# Robustness Experiment Evaluation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
file = "n1_RobustnessExperiment_2022-01-17_11-23-01.csv"


In [3]:
df_frame = pd.read_csv(file)
df_frame['makespan_std'] = df_frame['makespan'] - df_frame['makespan-optimum']
df_frame['makespan_uncertainty_std'] = df_frame['makespan_uncertainty'] - df_frame['makespan-optimum']
df_frame['makespan_uncertainty_baseline_delay'] = df_frame['makespan_uncertainty_std'] - df_frame['makespan_std']
list_ordering = {"RandomSolver": 0, "HillClimbing": 1, "TabuSearch": 2, "SimulatedAnnealing": 3, "GeneticAlgorithm": 4}
df_frame["solver"] = pd.Categorical(df_frame["solver"], categories=list_ordering)
df_frame.loc[~pd.notna(df_frame['robustness_measure']), 'robustness_measure'] = "No-RM"
df_frame.head(100)

,benchmark,solver,iterations,makespan-optimum,makespan,robustness,robustness_measure,uncertainty_percentage,makespan_uncertainty,robustness_uncertainty,makespan_std,makespan_uncertainty_std,makespan_uncertainty_baseline_delay
0,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,18,7.4,SF3_W1,0.0,18,7.4,1,1,0
1,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
2,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
3,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,18,7.4,SF3_W1,0.0,18,7.4,1,1,0
4,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
96,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,18,7.4,SF3_W1,0.0,18,7.4,1,1,0
97,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
98,n1.mm/n110_1.mm,GeneticAlgorithm,2500,17,18,7.4,SF3_W1,0.0,18,7.4,1,1,0


In [4]:
df_grouped = df_frame.groupby(['uncertainty_percentage', 'iterations', 'robustness_measure'])['makespan_uncertainty_baseline_delay'].describe()[['mean', 'std']]
df_grouped.head()

mean  std
uncertainty_percentage iterations robustness_measure           
0.0                    500        No-RM                0.0  0.0
                                  SF1                  0.0  0.0
                                  SF1_W1               0.0  0.0
                                  SF1_W9               0.0  0.0
                                  SF2                  0.0  0.0

In [5]:
# print(proactive_describe.to_latex(float_format="{:0.2f}".format))
#for solver in df_frame['solver'].unique():
group = df_grouped.reset_index()
 #   group = group[group['solver'] == solver]
group = group.set_index(['iterations', 'robustness_measure'])
group = group.pivot(columns='uncertainty_percentage')
print(group.to_latex(float_format="{:0.2f}".format))


\begin{tabular}{llrrrrrrrr}
\toprule
     & {} & \multicolumn{4}{l}{mean} & \multicolumn{4}{l}{std} \\
     & uncertainty\_percentage & 0.00 & 0.05 & 0.10 & 0.20 & 0.00 & 0.05 & 0.10 & 0.20 \\
iterations & robustness\_measure &      &      &      &      &      &      &      &      \\
\midrule
500  & No-RM & 0.00 & 0.86 & 1.66 & 3.18 & 0.00 & 1.06 & 1.40 & 1.90 \\
     & SF1 & 0.00 & 0.84 & 1.60 & 3.11 & 0.00 & 1.05 & 1.40 & 1.87 \\
     & SF1\_W1 & 0.00 & 0.83 & 1.61 & 3.08 & 0.00 & 1.08 & 1.45 & 1.86 \\
     & SF1\_W9 & 0.00 & 0.84 & 1.62 & 3.11 & 0.00 & 1.10 & 1.51 & 1.92 \\
     & SF2 & 0.00 & 0.85 & 1.65 & 3.16 & 0.00 & 1.16 & 1.56 & 2.02 \\
     & SF2\_W1 & 0.00 & 0.80 & 1.57 & 3.04 & 0.00 & 0.98 & 1.34 & 1.83 \\
     & SF2\_W9 & 0.00 & 0.81 & 1.59 & 3.05 & 0.00 & 1.06 & 1.42 & 1.82 \\
     & SF3 & 0.00 & 0.80 & 1.56 & 3.03 & 0.00 & 1.00 & 1.33 & 1.77 \\
     & SF3\_W1 & 0.00 & 0.81 & 1.58 & 3.06 & 0.00 & 1.01 & 1.41 & 1.82 \\
     & SF3\_W9 & 0.00 & 0.82 & 1.60 & 3.09 & 0.00 & 1.